In [1]:
import os
import folium
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Polygon

from utils import *

### Fountains

In [24]:
fountains = pd.read_csv(os.getenv("fountains"))
fountains.columns = fountains.columns.str.lower()

In [25]:
fountains = gpd.GeoDataFrame(fountains, geometry=gpd.points_from_xy(fountains.longitude, fountains.latitude))

In [26]:
print(fountains.crs)

None


In [27]:
fountains = fountains.set_crs('epsg:4326')

In [28]:
print(fountains.crs)

epsg:4326


In [29]:
columns = ["precision_localisation", "date_installation", "etat", "x", "y"]
fountains = fountains.drop(columns=columns)

In [30]:
fountains.head(1)

,id,arrondissement,nom_parc_lieu,proximité_jeux_repère,intersection,remarque,longitude,latitude,geometry
0,1,Saint-Léonard,Delorme,Bocce,rue Loranger/rue Péguy,NaN,-73.589462,45.592012,POINT (-73.58946 45.59201)


### Urban furniture

In [31]:
mobilier = gpd.read_file(os.getenv("mobilier"))
mobilier.columns = mobilier.columns.str.lower()

In [32]:
print(mobilier.crs)

EPSG:4326


In [33]:
mobilier = mobilier.to_crs("epsg:4326")

In [34]:
print(mobilier.crs)

EPSG:4326


In [35]:
columns = ["element", "inspectdat", "actif", "ancree", "instaldate", "materiau", "grarg",
           "grasec", "indexgparc", "typeancrag", "modele", "x", "y"]
mobilier = mobilier.drop(columns=columns)

In [36]:
mobilier.head(1)

,objectid,elementdes,utilisatio,remarque,nom_parc,long,lat,geometry
0,91,table a pique-nique (TPN),I-Intense,remisage de nombreuses tpn d'événements,PARC LA FONTAINE,-73.569832,45.528918,POINT (-73.56983 45.52892)


### calcul the colsest distance between fountain and urban furniture 

In [37]:
fountains_utm = fountains.to_crs(epsg=32188)
mobilier_utm = mobilier.to_crs(epsg=32188)

In [38]:
print(fountains_utm.crs)
print(mobilier_utm.crs)

EPSG:32188
EPSG:32188


In [39]:
fountain_w_furniture_data = fountains_utm.sjoin_nearest(mobilier_utm, distance_col="nearest_urban")

In [ ]:
# fountain_w_furniture_data['distances'] = fountain_w_furniture_data['distances'].map('{:.0f}'.format)

In [40]:
min(fountain_w_furniture_data["nearest_urban"])

0.0003687759228418947

In [41]:
max(fountain_w_furniture_data["nearest_urban"])

5141.165468421593

In [42]:
m = folium.Map(location=[45.5414195, -73.6303031], tiles="cartodbpositron", zoom_start=12)

In [43]:
part = fountain_w_furniture_data[fountain_w_furniture_data.nom_parc_lieu.str.contains("Royal", na=False)]
for _, row in part.iterrows():
    lat = row.latitude
    lon = row.longitude
    folium.Marker(location=[lat, lon], popup=row["id"]).add_to(m)

In [44]:
m

In [45]:
fountain_w_furniture_data.head(2)

,id,arrondissement,nom_parc_lieu,proximité_jeux_repère,intersection,remarque_left,longitude,latitude,geometry,index_right,objectid,elementdes,utilisatio,remarque_right,nom_parc,long,lat,nearest_urban
0,1,Saint-Léonard,Delorme,Bocce,rue Loranger/rue Péguy,NaN,-73.589462,45.592012,POINT (297819.890 5050237.940),5504,11226,bollard (BOL),N-Normal,NaN,PARC-NATURE DU RUISSEAU-DE MONTIGNY,-73.600301,45.613466,2529.702004
1,2,Saint-Léonard,Delorme,Pavillon,rue Loranger/rue Péguy,NaN,-73.590055,45.592189,POINT (297773.670 5050257.680),5504,11226,bollard (BOL),N-Normal,NaN,PARC-NATURE DU RUISSEAU-DE MONTIGNY,-73.600301,45.613466,2495.967421


In [46]:
part = fountain_w_furniture_data.copy()
for _, row in part.iterrows():
    lat = row.lat
    lon = row.long
    folium.CircleMarker(
        location=[lat, lon], 
        radius=2,
        color="red",
        fill=True,
        fill_opacity=0.7,
        fill_color="red",
        popup=row["objectid"]).add_to(m)

In [48]:
m

In [ ]:
# f_utm.sjoin_nearest(mb_utm, distance_col="distances")

In [51]:
part = fountain_w_furniture_data[fountain_w_furniture_data.nom_parc_lieu.str.contains("Royal", na=False)]

In [53]:
part

,id,arrondissement,nom_parc_lieu,proximité_jeux_repère,intersection,remarque_left,longitude,latitude,geometry,index_right,objectid,elementdes,utilisatio,remarque_right,nom_parc,long,lat,nearest_urban
620,631,Ville-Marie,près du parc du Mont-Royal,NaN,ave des Pins Ouest/rue Peel,au pied de l'escalier,-73.582003,45.504592,POINT (298391.930 5040522.260),2893,8130,poubelle (POU),I-Intense,NaN,PARC DU MONT-ROYAL,-73.582238,45.504685,21.056610
827,841,Ville-Marie,du Mont-Royal,chalet,NaN,Grands Parcs,-73.587419,45.503420,POINT (297968.620 5040392.460),1404,3797,fontaine a boire (FAB),I-Intense,adaptée,PARC DU MONT-ROYAL,-73.587419,45.503420,0.002487
828,842,Ville-Marie,du Mont-Royal,chalet,NaN,Grands Parcs,-73.586842,45.503720,POINT (298013.690 5040425.750),1405,3798,fontaine a boire (FAB),I-Intense,adaptée,PARC DU MONT-ROYAL,-73.586842,45.503720,0.001368
829,843,Ville-Marie,du Mont-Royal,sentier,NaN,Grands Parcs,-73.587095,45.502097,POINT (297993.790 5040245.380),1661,4125,fontaine a boire (FAB),N-Normal,NaN,PARC DU MONT-ROYAL,-73.587095,45.502097,0.006079
830,844,Ville-Marie,du Mont-Royal,chemin,NaN,Grands Parcs,-73.591708,45.502730,POINT (297633.330 5040316.150),1664,4129,fontaine a boire (FAB),I-Intense,NaN,PARC DU MONT-ROYAL,-73.591708,45.502730,0.008540
831,845,Ville-Marie,du Mont-Royal,chalet,NaN,Grands Parcs,-73.587162,45.504390,POINT (297988.780 5040500.190),1675,4140,fontaine a boire (FAB),I-Intense,avec pied en métal,PARC DU MONT-ROYAL,-73.587162,45.504390,0.004019
832,846,Ville-Marie,du Mont-Royal,chalet,NaN,Grands Parcs,-73.588005,45.504188,POINT (297922.910 5040477.850),1676,4141,fontaine a boire (FAB),I-Intense,non adaptée,PARC DU MONT-ROYAL,-73.588005,45.504188,0.006636
833,847,Ville-Marie,du Mont-Royal,chalet,près du lac des castors,Grands Parcs,-73.599150,45.498618,POINT (297051.250 5039859.870),1692,4157,fontaine a boire (FAB),I-Intense,adaptée,PARC DU MONT-ROYAL,-73.599150,45.498618,0.006383
834,848,Ville-Marie,du Mont-Royal,chemin,NaN,Grands Parcs,-73.589275,45.504000,POINT (297823.630 5040457.050),1841,4330,fontaine a boire (FAB),I-Intense,adaptée,PARC DU MONT-ROYAL,-73.589275,45.504000,0.001569
835,849,Ville-Marie,du Mont-Royal,sentier,près du lac des castors,Grands Parcs,-73.599359,45.498969,POINT (297034.910 5039898.870),2415,5200,fontaine a boire (FAB),I-Intense,non adaptée,PARC DU MONT-ROYAL,-73.599359,45.498969,0.004954
